# Feature Extraction

참조 : https://github.com/GyeongHwanJung/Instacart-Market-Basket-Analysis 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

!pip install category_encoders

import category_encoders as ce
import gc

PATH = '/content/drive/MyDrive/data/instacart-market-basket-analysis/' # csv파일이 들어있는 폴더 경로 지정

     |████████████████████████████████| 80 kB 5.0 MB/s 


In [ ]:
priors = pd.read_csv(PATH + 'order_products__prior.csv', dtype={
            'order_id': np.uint32,          # 1 ~ 3421083
            'product_id': np.uint16,        # 1 ~ 49688
            'add_to_cart_order': np.uint8,  # 1 ~ 80 
            'reordered': np.uint8})         # 0 ~ 1
orders = pd.read_csv(PATH + 'orders.csv', dtype={
        'order_id': np.uint32,              # 1 ~ 3421083
        'user_id': np.uint32,               # 1 ~ 206209
        'eval_set': 'category',             # 3가지
        'order_number': np.uint8,           # 1 ~ 100
        'order_dow': np.uint8,              # 0 ~ 6 
        'order_hour_of_day': np.uint8,      # 0 ~ 23
        'days_since_prior_order': np.float32})
products = pd.read_csv(PATH + 'products.csv', dtype={
        'product_id': np.uint16,     # 1 ~ 49688
        'aisle_id': np.uint8,        # 1 ~ 134
        'department_id': np.uint8}) # 1 ~ 21
aisles = pd.read_csv(PATH + 'aisles.csv')
departments = pd.read_csv(PATH + 'departments.csv')
train = pd.read_csv(PATH + 'order_products__train.csv')

### 데이터 타입 함수

In [ ]:
## 데이터 크기 확인 함수
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

## 이산형 데이터 사이즈 축소 함소
def int_memory_reduce(data) :
    data_int = data.select_dtypes(include=['int'])
    converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')
    print(f"Before : {mem_usage(data_int)} -> After : {mem_usage(converted_int)}")
    data[converted_int.columns] = converted_int
    return data

## 연속형 데이터 사이즈 축소 함소
def float_memory_reduce(data) :
    data_float = data.select_dtypes(include=['float'])
    converted_float = data_float.apply(pd.to_numeric,downcast='float')
    print(f"Before : {mem_usage(data_float)} -> After : {mem_usage(converted_float)}")
    data[converted_float.columns] = converted_float
    return data

## 문자형 데이터 사이즈 축소 함소
def object_memory_reduce(data) :
    gl_obj = data.select_dtypes(include=['object']).copy()
    converted_obj = pd.DataFrame()
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]
    print(f"Before : {mem_usage(gl_obj)} -> After : {mem_usage(converted_obj)}")
    data[converted_obj.columns] = converted_obj
    return data

def memory_reduce(data, except_col=None) :
    print(f"Before : {mem_usage(data)}")
    if except_col:
        col = data[except_col]
        data.drop(except_col, axis=1, inplace=True)

    data_int = data.select_dtypes(include=['int'])
    data_float = data.select_dtypes(include=['float'])
    gl_obj = data.select_dtypes(include=['object']).copy()

    converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')
    converted_float = data_float.apply(pd.to_numeric,downcast='float')
    converted_obj = pd.DataFrame()
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]
    
    print(f"정수형 데이터 Before : {mem_usage(data_int)} -> After : {mem_usage(converted_int)}")
    print(f"실수형 데이터 Before : {mem_usage(data_float)} -> After : {mem_usage(converted_float)}")
    print(f"문자열 데이터 Before : {mem_usage(gl_obj)} -> After : {mem_usage(converted_obj)}")
    
    data[converted_int.columns] = converted_int
    data[converted_float.columns] = converted_float
    data[converted_obj.columns] = converted_obj

    if except_col:
        data = pd.concat([data, col], axis=1)

    print(f"After : {mem_usage(data)}\n")

    return data

# 준비

- priors에 orders와 products 병합

priors는 마지막 주문을 제외한 모든 주문의 구매내역이다. 

orders는 모든 주문이 있다. 

products는 모든 제품이 있다. 

3개의 데이터프레임을 병합하면 다음과 같은 정보를 얻을 수 있다. 

1. priors에 orders를 병합 

    order_id를 기준으로 inner 병합을 하기 때문에 두 df의 교집합을 사용한다. priors는 마지막 주문이 제외되어 있기 때문에 orders의 마지막 주문 정보들은 사라지게 된다. 

2. priors에 products를 병합

    product_id를 기준으로 left 병합을 한다. priors의 product_id에 있는 행을 기준으로 병합한다.     
    priors에는 마지막 주문을 제외한 모든 구매내역이 있기 때문에 만약 구매내역이 있는 제품의 정보만 남게된다.  


In [ ]:
prior_df = priors.merge(orders, on='order_id' , how='inner')        
prior_df = prior_df.merge(products, on='product_id', how='left')  
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


# feature creation

유저가 해당 제품을 총 몇번 샀는지 누적합 피쳐 생성

In [ ]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [ ]:
prior_df[(prior_df.user_id == 202279) & (prior_df.product_id == 33120)].user_buy_product_times

0           1
990781      2
1254949     3
26635932    4
27453973    5
Name: user_buy_product_times, dtype: int64

만든 피쳐를 한 번 확인해보면 202279 유저가 33120 제품을 구매한 횟수는 총 5번이다. 

## product features


product로 묶어서 만들 수 있는 피쳐는 뭐가 있을까? (제품별)

- add_to_cart_order : 평균 재주문율
- reordered : 총 주문된 수, 총 재주문 수, 재주문율
- user_id : 이 제품을 구매한 유저 수, 첫번째 구매의 수, 두번째 구매의 수

    첫번째 구매 수랑 두번째 구매 수가 중요한 이유는 두 주문의 차이를 보면 인사이트를 얻을 수 있다. 두 수의 차이가 작다면 재주문한 유저가 많다는 것을 알 수 있다. 반대로 차이가 크다면 재주문한 유저의 수가 적다는 것을 알 수 있다.  

- product_name : organic 여부
- 두번째 주문율 : 두번째 주문의 합을 첫번째 주문의 합으로 나눠주면 얼마나 재구매 했는지 알 수 있다.
- 판매패턴? : 가장 많이 팔린 시간, 요일, 재주문 기간

In [ ]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [ ]:
# 먼저 컬럼을 지정하고 : 뭉칠 때 쓰는 함수를 선언해서 미리 조건을 다 정해둔다.
# 새로운 컬럼명은 리스트로 만들어 둔다.
# 주의사항! : 딕셔너리에 넣어둔 함수의 순서는 변환을 거치면 지정한 순서에서 바뀔 수 있어서 꼭 확인해 볼 것.
agg_dict1 = {'add_to_cart_order' : 'mean', 
             'reordered' : {'mean', 'count', 'sum'},
             'user_id' : 'nunique', 
             'user_buy_product_times' : {lambda x : sum(x == 1), lambda x : sum(x == 2)}, 
             'product_name' : lambda x: 1 if 'Organic' in x else 0}

col1 = ['mean_add_to_cart_order', 'reorder_rate', 
        'total_orders', 'total_reorders',
       'unique_users', 'order_1st_total_cnt', 
       'order_2nd_total_cnt', 'is_organic']

In [ ]:
prod_feats = prior_df.groupby('product_id').agg(agg_dict1)
prod_feats.columns = col1
prod_feats.reset_index(inplace=True)
prod_feats['order_2nd_rate'] = prod_feats.order_2nd_total_cnt / prod_feats.order_1st_total_cnt
prod_feats.head()

,product_id,mean_add_to_cart_order,reorder_rate,total_orders,total_reorders,unique_users,order_1st_total_cnt,order_2nd_total_cnt,is_organic,order_2nd_rate
0,1,5.801836,0.613391,1852,1136.0,716,716,276,0,0.385475
1,2,9.888889,0.133333,90,12.0,78,78,8,0,0.102564
2,3,6.415162,0.732852,277,203.0,74,74,36,0,0.486486
3,4,9.507599,0.446809,329,147.0,182,182,64,0,0.351648
4,5,6.466667,0.600000,15,9.0,6,6,4,0,0.666667


### aisle and department features

- add_to_cart_order : 평균 순위, 표준편차 순위
- reorderd : 총 판매수, 재구매 수, 재구매 율
- user_id : 구매 유저 수

In [ ]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [ ]:
agg_dict2 = {'add_to_cart_order' : {'mean','std'}, 
           'reordered' : {'mean', 'count', 'sum'},
           'user_id': 'nunique'}

col2 = ['aisle_mean_add_to_cart_order', 'aisle_std_add_to_cart_order', 
        'aisle_reorder_percentage', 'aisle_total_orders', 
        'aisle_total_reorders', 'aisle_unique_users']

In [ ]:
aisle_feats = prior_df.groupby('aisle_id').agg(agg_dict2)
aisle_feats.columns = col2
aisle_feats.reset_index(inplace = True)
aisle_feats.head()

,aisle_id,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_reorder_percentage,aisle_total_orders,aisle_total_reorders,aisle_unique_users
0,1,8.167640,7.104166,0.596597,71928,42912.0,20711
1,2,9.275497,7.473802,0.489326,82491,40365.0,31222
2,3,9.571935,7.899672,0.598007,456386,272922.0,63592
3,4,10.161450,7.745705,0.489533,200687,98243.0,53892
4,5,10.297600,8.187047,0.280627,62510,17542.0,32312


In [ ]:
agg_dict3 = {'add_to_cart_order' : {'mean','std'}, 
           'reordered' : {'mean', 'count', 'sum'},
           'user_id': 'nunique'}

col3 = ['department_mean_add_to_cart_order', 'department_std_add_to_cart_order', 
        'department_reorder_percentage', 'department_total_orders', 
        'department_total_reorders', 'department_unique_users']

In [ ]:
dpt_feats = prior_df.groupby('department_id').agg(agg_dict2)
dpt_feats.columns = col3
dpt_feats.reset_index(inplace = True)
dpt_feats.head()

,department_id,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_reorder_percentage,department_total_orders,department_total_reorders,department_unique_users
0,1,8.996414,7.393502,0.541885,2236432,1211890.0,163233
1,2,8.277645,7.526272,0.407980,36291,14806.0,17875
2,3,8.084397,6.904849,0.628141,1176787,739188.0,140612
3,4,8.022875,6.658899,0.649913,9479291,6160710.0,193237
4,5,5.428346,5.778253,0.569924,153696,87595.0,15798


### prod_feats에 병합

In [ ]:
prod_feats = prod_feats.merge(products, on = 'product_id', how = 'left')
prod_feats = prod_feats.merge(aisle_feats, on = 'aisle_id', how = 'left')
prod_feats = prod_feats.merge(aisles, on = 'aisle_id', how = 'left')
prod_feats = prod_feats.merge(dpt_feats, on = 'department_id', how = 'left')
prod_feats = prod_feats.merge(departments, on = 'department_id', how = 'left')
print(prod_feats.shape)
prod_feats.head()

(49677, 27)


,product_id,mean_add_to_cart_order,reorder_rate,total_orders,total_reorders,unique_users,order_1st_total_cnt,order_2nd_total_cnt,is_organic,order_2nd_rate,...,aisle_total_reorders,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_reorder_percentage,department_total_orders,department_total_reorders,department_unique_users,department
0,1,5.801836,0.613391,1852,1136.0,716,716,276,0,0.385475,...,128431.0,54202,cookies cakes,9.187743,7.692492,0.574180,2887550,1657973.0,174219,snacks
1,2,9.888889,0.133333,90,12.0,78,78,8,0,0.102564,...,32321.0,76402,spices seasonings,9.593425,7.875241,0.346721,1875577,650301.0,172755,pantry
2,3,6.415162,0.732852,277,203.0,74,74,36,0,0.486486,...,131556.0,53197,tea,6.976699,6.711172,0.653460,2690129,1757892.0,172795,beverages
3,4,9.507599,0.446809,329,147.0,182,182,64,0,0.351648,...,217262.0,58749,frozen meals,8.996414,7.393502,0.541885,2236432,1211890.0,163233,frozen
4,5,6.466667,0.600000,15,9.0,6,6,4,0,0.666667,...,17542.0,32312,marinades meat preparation,9.593425,7.875241,0.346721,1875577,650301.0,172755,pantry


In [ ]:
prod_feats.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
print(prod_feats.shape)
prod_feats.head()

(49677, 24)


,product_id,mean_add_to_cart_order,reorder_rate,total_orders,total_reorders,unique_users,order_1st_total_cnt,order_2nd_total_cnt,is_organic,order_2nd_rate,...,aisle_total_reorders,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_reorder_percentage,department_total_orders,department_total_reorders,department_unique_users,department
0,1,5.801836,0.613391,1852,1136.0,716,716,276,0,0.385475,...,128431.0,54202,cookies cakes,9.187743,7.692492,0.574180,2887550,1657973.0,174219,snacks
1,2,9.888889,0.133333,90,12.0,78,78,8,0,0.102564,...,32321.0,76402,spices seasonings,9.593425,7.875241,0.346721,1875577,650301.0,172755,pantry
2,3,6.415162,0.732852,277,203.0,74,74,36,0,0.486486,...,131556.0,53197,tea,6.976699,6.711172,0.653460,2690129,1757892.0,172795,beverages
3,4,9.507599,0.446809,329,147.0,182,182,64,0,0.351648,...,217262.0,58749,frozen meals,8.996414,7.393502,0.541885,2236432,1211890.0,163233,frozen
4,5,6.466667,0.600000,15,9.0,6,6,4,0,0.666667,...,17542.0,32312,marinades meat preparation,9.593425,7.875241,0.346721,1875577,650301.0,172755,pantry


인코딩

컬럼의 조합으로 데이터를 표현한다. 원 핫 인코딩보다 훨씬 효율적이라고 생각된다.

In [ ]:
encoder = ce.BinaryEncoder(cols=['aisle', 'department'], return_df=True)
prod_feats = encoder.fit_transform(prod_feats)
prod_feats.head()

,product_id,mean_add_to_cart_order,reorder_rate,total_orders,total_reorders,unique_users,order_1st_total_cnt,order_2nd_total_cnt,is_organic,order_2nd_rate,...,department_std_add_to_cart_order,department_reorder_percentage,department_total_orders,department_total_reorders,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,0.613391,1852,1136.0,716,716,276,0,0.385475,...,7.692492,0.574180,2887550,1657973.0,174219,0,0,0,0,1
1,2,9.888889,0.133333,90,12.0,78,78,8,0,0.102564,...,7.875241,0.346721,1875577,650301.0,172755,0,0,0,1,0
2,3,6.415162,0.732852,277,203.0,74,74,36,0,0.486486,...,6.711172,0.653460,2690129,1757892.0,172795,0,0,0,1,1
3,4,9.507599,0.446809,329,147.0,182,182,64,0,0.351648,...,7.393502,0.541885,2236432,1211890.0,163233,0,0,1,0,0
4,5,6.466667,0.600000,15,9.0,6,6,4,0,0.666667,...,7.875241,0.346721,1875577,650301.0,172755,0,0,0,1,0


In [ ]:
prod_feats[['department_0', 'department_1', 'department_2', 'department_3','department_4']]

,department_0,department_1,department_2,department_3,department_4
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
49672,1,0,1,0,1
49673,1,0,1,0,1
49674,1,0,1,0,1
49675,1,0,1,0,1


In [ ]:
# free some memory
del aisle_feats, dpt_feats, aisles, departments
gc.collect()

172

## user features

- 요일 : 평균, 표준편차
- 시간 : 평균, 표준편차
- 재주문 기간 : 평균, 표준편차
- 총 주문 수
- 총 구매 제품 수, 총 구매 제품 종류 수
- 재주문 제품 수, 평균 재주문 제품 수

In [ ]:
agg_dict4 = {'order_dow': {'mean', 'std'},
           'order_hour_of_day': {'mean', 'std'},
           'days_since_prior_order': {'mean', 'std'},
           'order_number': 'nunique',
           'product_id': {'nunique', 'count'},
           'reordered': {'mean', 'sum'}}

col4 = ['avg_dow', 'std_dow', 'avg_doh', 'std_doh', 
        'avg_since_order', 'std_since_order', 'total_orders_by_user', 
        'total_unique_product_by_user', 'total_products_by_user',  
        'reorder_propotion_by_user', 'total_reorders_by_user']

In [ ]:
# 유저 nunique 
user_feats = prior_df.groupby('user_id').agg(agg_dict4)
user_feats.columns = col4
user_feats.reset_index(inplace = True)
print(user_feats.shape)
user_feats.head()

(206209, 12)


,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_unique_product_by_user,total_products_by_user,reorder_propotion_by_user,total_reorders_by_user
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,18,59,0.694915,41.0
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,102,195,0.476923,93.0
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,33,88,0.625000,55.0
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,17,18,0.055556,1.0
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,23,37,0.378378,14.0


- reorder_in_order : 해당 주문의 재주문 제품 비율
- average_order_size : 주문 별 구매 제품 수


In [ ]:
# 유저와 유저의 주문의 조합
agg_dict5 = {'reordered': {'mean', 'count'}}
col5 = ['reorder_in_order', 'average_order_size']

user_feats2 = prior_df.groupby(['user_id', 'order_number']).agg(agg_dict5)
user_feats2.columns = col5
user_feats2.reset_index(inplace = True)
print(user_feats2.shape)
user_feats2.head()

(3214874, 4)


,user_id,order_number,reorder_in_order,average_order_size
0,1,1,0.000,5
1,1,2,0.500,6
2,1,3,0.600,5
3,1,4,1.000,5
4,1,5,0.625,8


- reorder_in_order : 해당 유저의 모든 주문의 재주문 제품 비율
- average_order_size : 평균 구매 제품 수

In [ ]:
# 유저 & 주문 조합에서 유저 nunique로 뽑아낸다. 
user_feats3 = user_feats2.groupby('user_id').agg({'average_order_size':'mean', 
                                                  'reorder_in_order':'mean'})
user_feats3 = user_feats3.reset_index()
print(user_feats3.shape)
user_feats3.head()

(206209, 3)


,user_id,average_order_size,reorder_in_order
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


병합

In [ ]:
user_feats = user_feats.merge(user_feats3, on = 'user_id', how = 'left')
print(user_feats.shape)
user_feats.head()

(206209, 14)


,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_unique_product_by_user,total_products_by_user,reorder_propotion_by_user,total_reorders_by_user,average_order_size,reorder_in_order
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,18,59,0.694915,41.0,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,102,195,0.476923,93.0,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,33,88,0.625000,55.0,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,17,18,0.055556,1.0,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,23,37,0.378378,14.0,9.250000,0.377778


In [ ]:
# 유저별로 마지막 3개의 주문 순서를 가져온다. 유저의 최소 주문수가 4여서 맞춰야하는 마지막 주문을 제외하고 나머지 주문이 최대 3이다. 
# 3분 걸림 ;; 
last_three_orders = user_feats2.groupby('user_id')['order_number'].nlargest(3).reset_index()
print(last_three_orders.shape)
last_three_orders.head()

(618627, 3)


,user_id,level_1,order_number
0,1,9,10
1,1,8,9
2,1,7,8
3,2,23,14
4,2,22,13


마지막 3개의 주문만 사용한다.

In [ ]:
# 유저 & 모든 주문에 마지막 3개의 주문을 병합하면 마지막 3개 주문의 정보만 뽑을 수 있다. 
last_three_orders = user_feats2.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
print(last_three_orders.shape)
last_three_orders.head()

(618627, 5)


,user_id,order_number,reorder_in_order,average_order_size,level_1
0,1,8,0.666667,6,7
1,1,9,1.000000,6,8
2,1,10,0.666667,9,9
3,2,12,0.578947,19,21
4,2,13,0.000000,9,22


- rank : 마지막 3개의 주문의 순서대로 rank를 준다. 피벗을 할 때 컬럼명으로 사용하기 위해서 생성.  

In [ ]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank()
print(last_three_orders.shape)
last_three_orders.head()

(618627, 6)


,user_id,order_number,reorder_in_order,average_order_size,level_1,rank
0,1,8,0.666667,6,7,1.0
1,1,9,1.000000,6,8,2.0
2,1,10,0.666667,9,9,3.0
3,2,12,0.578947,19,21,1.0
4,2,13,0.000000,9,22,2.0


유저와 마지막 3개의 주문을 행으로 묶은 df를 피벗해서 유저id nunique 행을 가진 df로 변환한다. 컬럼은 마지막 3개의 주문에 대해 구매 제품 수와 재주문 제품 비율이다. 

In [ ]:
# 마지막 주문이 1이다. 슬라이싱할 때 [-1]느낌
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['average_order_size', 'reorder_in_order']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','prod_order_3', 'prod_order_2', 'prod_order_1', 're_in_order_3', 're_in_order_2', 're_in_order_1']
print(last_order_feats.shape)
last_order_feats.head()

(206209, 7)


,user_id,prod_order_3,prod_order_2,prod_order_1,re_in_order_3,re_in_order_2,re_in_order_1
0,1,6,6,9,0.666667,1.0,0.666667
1,2,19,9,16,0.578947,0.0,0.625000
2,3,6,5,6,0.833333,1.0,1.000000
3,4,7,2,3,0.142857,0.0,0.000000
4,5,9,5,12,0.444444,0.4,0.666667


이제 users df에 병합한다. 

In [ ]:
user_feats = user_feats.merge(last_order_feats, on = 'user_id', how = 'left')
print(user_feats.shape)
user_feats.head()

(206209, 20)


,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_unique_product_by_user,total_products_by_user,reorder_propotion_by_user,total_reorders_by_user,average_order_size,reorder_in_order,prod_order_3,prod_order_2,prod_order_1,re_in_order_3,re_in_order_2,re_in_order_1
0,1,2.644068,1.256194,10.542373,3.500355,20.259260,9.304463,10,18,59,0.694915,41.0,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,102,195,0.476923,93.0,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,11.487180,4.869048,12,33,88,0.625000,55.0,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,15.357142,8.580901,5,17,18,0.055556,1.0,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,23,37,0.378378,14.0,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [ ]:
user_feats.head(3).T

,0,1,2
user_id,1.000000,2.000000,3.000000
avg_dow,2.644068,2.005128,1.011364
std_dow,1.256194,0.971222,1.245630
avg_doh,10.542373,10.441026,16.352273
std_doh,3.500355,1.649854,1.454599
avg_since_order,20.259260,15.967033,11.487180
std_since_order,9.304463,9.119769,4.869048
total_orders_by_user,10.000000,14.000000,12.000000
total_unique_product_by_user,18.000000,102.000000,33.000000
total_products_by_user,59.000000,195.000000,88.000000


유저별로 총 23개의 피쳐를 생성했다. 

- 주문 요일 : 평균, 표준편차
- 주문 시간 : 평균, 표준편차
- 재주문 기간 : 평균, 표준편차
- 총 주문 수
- 총 구매 제품 수 (중복 포함)
- 총 구매 제품 종류 수
- **reorder_propotion_by_user : 유저가 구매한 전체 제품 중 재주문 제품 비율**
- 재주문 제품 수
- 주문 별 평균 제품 수
- **reorder_in_order : 주문 별 재주문 제품의 비율을 다시 전체 주문수로 평균을 낸 수. 전체 주문에서 재주문 제품이 포함된 주문의 비율을 알 수 있다.** 

    1번 유저는 10번 주문했음. 총 20개의 제품 구매했음. 20개 중 재구매 제품은 10개임. reorder_propotion_by_user는`10/20 == 0.5`다.

    1~8번의 주문동안 1씩 모두 다른 종류를 구매. 9번째는 2개 구매. 다 처음 구매 제품. (지금까지 reordered == 0). 마지막에 총 11개의 제품을 구매하면서 10개의 제품은 재주문 제품이다. 이 경우 reorder_in_order는`(10/11)/10 == 0.09`다.

- 뒤에서 세번째까지 주문 별 구매 제품 수와 재주문 제품 비율 


## user and product level features



유저와 유저가 구매한 모든 제품을 1:1로 매칭한다.

- 해당 제품의 재구매율, 재구매수, 총 구매수 (reordered를 가지고 뽑기 빼문에 총 구매수-1이 재구매수이다.)
- 해당 제품의 평균 장바구니 순위
- 해당 제품의 평균 재구매 기간
- 해당 유저의 전체 주문 중 해당 제품의 마지막 주문 순서 (예를들어 총 주문은 10인데 이 제품의 마지막 주문은 8번째라는 것을 알 수 있다.) 

In [ ]:
agg_dict6 = {'reordered': {'mean', 'count', 'sum', },
            'add_to_cart_order': 'mean',
            'days_since_prior_order': 'mean',
            'order_number': 'max'}
col6 = ['user_product_reorder_percentage', 'total_product_orders_by_user', 
        'total_product_reorders_by_user', 'avg_add_to_cart_by_user', 
        'avg_days_since_last_bought', 'last_ordered_in']

In [ ]:
user_product_feats = prior_df.groupby(['user_id', 'product_id']).agg(agg_dict6)
user_product_feats.columns = col6
user_product_feats.reset_index(inplace = True)
print(user_product_feats.shape)
user_product_feats.head()

(13307953, 8)


,user_id,product_id,user_product_reorder_percentage,total_product_orders_by_user,total_product_reorders_by_user,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in
0,1,196,0.900000,10,9,1.400000,19.555555,10
1,1,10258,0.888889,9,8,3.333333,19.555555,10
2,1,10326,0.000000,1,0,5.000000,28.000000,5
3,1,12427,0.900000,10,9,3.300000,19.555555,10
4,1,13032,0.666667,3,2,6.333333,21.666666,10


유저별 마지막 3개의 주문 내역

priors에 last_three_orders를 병합한다. user_id와 order_number를 기준으로 inner병합을 한다. 

고객별로 마지막 3개의 주문만 남게된다.

In [41]:
last_orders = prior_df.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
print(last_orders.shape)
last_orders.head()

(6349201, 18)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times,reorder_in_order,average_order_size,level_1,rank
0,7,34050,1,0,142903,prior,11,2,14,30.0,Orange Juice,31,7,1,0.000000,2,2231251,2.0
1,7,46802,2,0,142903,prior,11,2,14,30.0,Pineapple Chunks,116,1,1,0.000000,2,2231251,2.0
2,14,20392,1,1,18194,prior,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,0.818182,11,282882,1.0
3,14,27845,2,1,18194,prior,49,3,15,3.0,Organic Whole Milk,84,16,1,0.818182,11,282882,1.0
4,14,162,3,1,18194,prior,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,0.818182,11,282882,1.0


피벗에 필요한 컬럼을 생성하기 위해서 rank컬럼 생성

마지막 3개의 주문 내역에서 유저별로 제품을 묶어서 주문 순서에 따라 순위를 매긴다. 주문이 3개니까 만약 유저가 제품 1을 3개의 주문에서 모두 구매했을 경우 주문 순서별로 순위가 생긴다.
만약 한 번 만 구매했다면 피쳐가 1개밖에 생성되지 않는다. 

In [42]:
last_orders['rank'] = last_orders.groupby(['user_id', 'product_id'])['order_number'].rank()
last_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times,reorder_in_order,average_order_size,level_1,rank
0,7,34050,1,0,142903,prior,11,2,14,30.0,Orange Juice,31,7,1,0.000000,2,2231251,1.0
1,7,46802,2,0,142903,prior,11,2,14,30.0,Pineapple Chunks,116,1,1,0.000000,2,2231251,1.0
2,14,20392,1,1,18194,prior,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,0.818182,11,282882,1.0
3,14,27845,2,1,18194,prior,49,3,15,3.0,Organic Whole Milk,84,16,1,0.818182,11,282882,1.0
4,14,162,3,1,18194,prior,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,0.818182,11,282882,1.0


In [ ]:
last_orders.pivot_table(index = ['user_id', 'product_id'], columns='rank', values = 'reordered').reset_index().head(10)

rank,user_id,product_id,1.0,2.0,3.0
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,NaN,NaN
4,1,25133,1.0,1.0,1.0
5,1,35951,0.0,NaN,NaN
6,1,38928,0.0,NaN,NaN
7,1,39657,0.0,NaN,NaN
8,1,46149,0.0,1.0,1.0
9,1,49235,0.0,1.0,NaN


유저 & 제품 형태로 피벗

유저와 제품을 1:1로 잡고 rank를 컬럼, reordered를 value로 피벗을 한다. 
한 행에는 유저와 제품이 1:1로 있고 1,2,3 컬럼별로 뒤에서 3번째, 2번째, 마지막 주문 순으로 해당 제품의 재주문 여부가 담겨있다. 
- 모든 컬럼의 값이 1이라면 해당 제품은 마지막 3번 이전부터 구매했던 제품이고 마지막 3번의 주문동안에도 모두 구매한 제품이라는 뜻이다. 유저의 선호도가 매우 높다고 할 수 있다. 
- 0/1/1 이라는 값을 갖는 행은 뒤에서 3번째 주문에서 처음 구매한 제품이고 그 이후 2번 동안 재주문했다. 구매해보고 만족했다고 해석할 수 있다. 
- 0/NaN/NaN 은 뒤에서 3번째 주문에서 처음 구매했지만 그 이후로 재주문을 하지 않았다는 뜻이다. 만족스럽지 않았다고 해석할 수 있다. 

In [43]:
product_purchase_history = last_orders.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id', 'product_id', 'order_-3', 'order_-2', 'order_-1']
print(product_purchase_history.shape)
product_purchase_history.head()

(4925215, 5)


,user_id,product_id,order_-3,order_-2,order_-1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,NaN,NaN
4,1,25133,1.0,1.0,1.0


유저 & 제품 df에 병합

유저와 유저가 구매한 모든 제품이 있는 user_product_feats에 마지막 3개 주문에서 구매한 제품 정보를 병합한다. `left`병합으로 유저가 구매한 모든 제품을 기준으로 마지막 3번의 주문에서 구매된 적 없는 제품은 결측치로 채워진다. 결측치로 남겨두는 것이 의미있어 보인다. 0으로 채우면 첫구매 여부와 겹쳐서 데이터가 흐려질 수 있을 것 같다.

모델을 돌려보면서 확인해본다.

In [44]:
user_product_feats = user_product_feats.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
print(user_product_feats.shape)
user_product_feats.head()

(13307953, 11)


,user_id,product_id,user_product_reorder_percentage,total_product_orders_by_user,total_product_reorders_by_user,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,order_-3,order_-2,order_-1
0,1,196,0.900000,10,9,1.400000,19.555555,10,1.0,1.0,1.0
1,1,10258,0.888889,9,8,3.333333,19.555555,10,1.0,1.0,1.0
2,1,10326,0.000000,1,0,5.000000,28.000000,5,NaN,NaN,NaN
3,1,12427,0.900000,10,9,3.300000,19.555555,10,1.0,1.0,1.0
4,1,13032,0.666667,3,2,6.333333,21.666666,10,1.0,NaN,NaN


In [45]:
user_product_feats.isnull().sum()

user_id                                   0
product_id                                0
user_product_reorder_percentage           0
total_product_orders_by_user              0
total_product_reorders_by_user            0
avg_add_to_cart_by_user                   0
avg_days_since_last_bought           868069
last_ordered_in                           0
order_-3                            8382738
order_-2                           12178443
order_-1                           13013477
dtype: int64

# Saving all features

In [46]:
prod_feats.to_pickle(PATH + 'product_features.pkl')
user_feats.to_pickle(PATH +'user_features.pkl')
user_product_feats.to_pickle(PATH +'user_product_features.pkl')

In [47]:
# 확인
# df = pd.read_pickle(PATH +'product_features.pkl')
# df.head()

,product_id,mean_add_to_cart_order,reorder_rate,total_orders,total_reorders,unique_users,order_1st_total_cnt,order_2nd_total_cnt,is_organic,order_2nd_rate,...,department_std_add_to_cart_order,department_reorder_percentage,department_total_orders,department_total_reorders,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,0.613391,1852,1136.0,716,716,276,0,0.385475,...,7.692492,0.574180,2887550,1657973.0,174219,0,0,0,0,1
1,2,9.888889,0.133333,90,12.0,78,78,8,0,0.102564,...,7.875241,0.346721,1875577,650301.0,172755,0,0,0,1,0
2,3,6.415162,0.732852,277,203.0,74,74,36,0,0.486486,...,6.711172,0.653460,2690129,1757892.0,172795,0,0,0,1,1
3,4,9.507599,0.446809,329,147.0,182,182,64,0,0.351648,...,7.393502,0.541885,2236432,1211890.0,163233,0,0,1,0,0
4,5,6.466667,0.600000,15,9.0,6,6,4,0,0.666667,...,7.875241,0.346721,1875577,650301.0,172755,0,0,0,1,0
